In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [2]:
# Carga de datos
df = pd.read_csv(r"../data/final/1_panel/final_dataset.csv")
df.insert(0, 'status_inf', df.pop('status_inf'))
df = df.drop(columns=['conglome', 'vivienda','hogar','codperso','periodo','ocupinf_t1','ocupinf_t2','periodo'])
df = df.dropna().reset_index(drop=True)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24232 entries, 0 to 24231
Data columns (total 72 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   status_inf               24232 non-null  int64  
 1   dpto                     24232 non-null  int64  
 2   dominio                  24232 non-null  int64  
 3   tuvotrabajo              24232 non-null  int64  
 4   sector_trabajador        24232 non-null  int64  
 5   categoria_trabajador     24232 non-null  int64  
 6   trabajopara              24232 non-null  int64  
 7   registropersonajuridica  24232 non-null  float64
 8   cuentassunat             24232 non-null  float64
 9   pagosueldo               24232 non-null  int64  
 10  pagosalario              24232 non-null  float64
 11  pagocomision             24232 non-null  float64
 12  pagodestajo              24232 non-null  float64
 13  pagosubvencion           24232 non-null  float64
 14  pagohonorarios        

In [ ]:
categoricas = ["dpto","dominio","tuvotrabajo","sector_trabajador","categoria_trabajador","trabajopara","registropersonajuridica","cuentassunat",
               "pagosueldo","pagosalario","pagocomision","pagodestajo","pagosubvencion","pagohonorarios","pagoganancianegocio","pagoagropecuario",
               "pagopropina","pagoespecie","pagootros","pagonorecibe","tipocontrato","numpersonastrabajo","tuvootrotrabajo","normtrabaja","disponiblehorastrabajar",
               "sexo","estadocivil","lenguamaterna","niveleduc","asiste_educ","usointernet","padece_enfer","sintoma_malestar","enfermedad","recaida","accidente",
               "notuvoenf","essalud","seguroprivado","eps","seguroffaa","sis","tipovivienda","materialparedes","materialpisos","materialtechos","vivienda_status",
               "viviendatitulo","agua_procedencia","conexionsshh","electricidad","combustible","celular","internet","pobreza","region","area","afp","onp","ciiu_6c"]

df_indep = df.drop(columns = ['status_inf'])
df_categoricas = df_indep[categoricas]
df_continuas = df_indep.drop(columns=categoricas)

print("Total de variables indep")
print(df_indep.shape[1])
print("La cantidad de variables indep categoricas:")
print(df_categoricas.shape[1])
print("La cantidad de variables indep continuas:")
print(df_continuas.shape[1])

Total de variables indep
71
La cantidad de variables indep categoricas:
60
La cantidad de variables indep continuas:
11


In [5]:
# Crear dummies a partir de las variables categóricas
df_categoricas_encoded = pd.get_dummies(df_categoricas, columns=categoricas, drop_first=True, dummy_na=False)
print('La cantidad de variables indep categóricas después de haberse convertido en dummies es:')
print(df_categoricas_encoded.shape[1])

print("La cantidad de variables indep continuas:")
print(df_continuas.shape[1])

# El dataset final de variables independientes (categoricas convertidas a dummies + continuas)
df_indep_final = pd.concat([df_categoricas_encoded, df_continuas], axis=1)
print('La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:')
print(df_indep_final.shape[1])

La cantidad de variables indep categóricas después de haberse convertido en dummies es:
189
La cantidad de variables indep continuas:
11
La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:
200


In [6]:
# Filtrar variables con un gran número de faltantes
null_percent = df_indep_final.isnull().mean()*100
cols_null = null_percent[null_percent > 80 ].index
cols_validas = null_percent[null_percent <= 80 ].index
print(cols_null)
# df_null = df[cols_validas].copy()
# df_null.shape[1]

Index([], dtype='object')


In [7]:
# Para eliminar las variables con muy poca varianza (definido por el umbral)
# Asegurar que todas las columnas dummy sean numéricas
df_indep_final = df_indep_final.apply(pd.to_numeric)

varianzas = df_indep_final.var()
umbral = 0.01
vars_seleccionadas = varianzas[varianzas > umbral].index
vars_no_seleccionadas = varianzas[varianzas <= umbral].index

print("Las variables con varianza menor al umbral son:")
print(df_indep_final[vars_no_seleccionadas].nunique())
## print(df_indep_final[vars_seleccionadas].nunique())

df_indep_final=df_indep_final[vars_seleccionadas]

print("El nuevo número de variables del dataset de variables independientes es:")
print(df_indep_final.shape[1])

Las variables con varianza menor al umbral son:
categoria_trabajador_6          2
trabajopara_3                   2
registropersonajuridica_99.0    2
pagocomision_1.0                2
pagodestajo_1.0                 2
pagohonorarios_1.0              2
pagoagropecuario_1.0            2
pagootros_1.0                   2
pagonorecibe_1.0                2
tipocontrato_3.0                2
tipocontrato_4.0                2
tipocontrato_5.0                2
lenguamaterna_3.0               2
lenguamaterna_5.0               2
lenguamaterna_6.0               2
lenguamaterna_7.0               2
lenguamaterna_8.0               2
tipovivienda_5.0                2
tipovivienda_6.0                2
tipovivienda_7.0                2
materialparedes_2.0             2
materialparedes_6.0             2
materialparedes_8.0             2
materialpisos_7.0               2
materialtechos_6.0              2
materialtechos_7.0              2
materialtechos_8.0              2
vivienda_status_5               2


In [8]:
import numpy as np

# Calcular la matriz de correlación
corr_matrix = df_indep_final.corr().abs()

# Seleccionar la parte superior de la matriz de correlación (sin la diagonal)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identificar las columnas a eliminar por correlación alta
to_drop = []
correlations_to_show = []

for column in upper.columns:
    high_corr = upper[column] > 0.70  # Umbral de 0.70
    if any(high_corr):
        to_drop.append(column)
        correlations_to_show.append((column, upper[column][high_corr]))

# Mostrar las correlaciones altas y las variables a eliminar
print("Correlaciones altas que exceden el umbral de 0.70:")
for column, corr_values in correlations_to_show:
    print(f"\nColumna: {column}")
    print(corr_values)

print("\nVariables a eliminar por alta correlación:")
print(to_drop)

# Eliminar las variables con alta correlación
df_indep_final = df_indep_final.drop(columns=to_drop)

# Mostrar el nuevo número de variables después de la eliminación
print("\nNuevo número de variables del dataset de variables independientes después de eliminar las que tienen una alta correlación:")
print(df_indep_final.shape[1])

Correlaciones altas que exceden el umbral de 0.70:

Columna: dominio_2
dpto_11    0.725462
Name: dominio_2, dtype: float64

Columna: dominio_8
dpto_15    0.71887
Name: dominio_8, dtype: float64

Columna: trabajopara_99
categoria_trabajador_2    0.769422
Name: trabajopara_99, dtype: float64

Columna: registropersonajuridica_2.0
categoria_trabajador_2    0.716144
trabajopara_99            0.809704
Name: registropersonajuridica_2.0, dtype: float64

Columna: registropersonajuridica_3.0
trabajopara_2    0.930537
Name: registropersonajuridica_3.0, dtype: float64

Columna: cuentassunat_2.0
trabajopara_99                 0.710342
registropersonajuridica_2.0    0.742730
Name: cuentassunat_2.0, dtype: float64

Columna: cuentassunat_99.0
trabajopara_2                  0.923276
registropersonajuridica_3.0    0.992196
Name: cuentassunat_99.0, dtype: float64

Columna: pagosueldo_1
categoria_trabajador_3    0.984918
Name: pagosueldo_1, dtype: float64

Columna: pagosalario_1.0
categoria_trabajador_4  

In [9]:
df_model_xgb_rf = pd.concat([df[["status_inf"]], df_indep_final], axis=1)

df_model_xgb_rf.to_csv(r'../data/final/1_panel/3_modelling/df_model_post_tratamiento_estadistico.csv', index=False)
